In [1]:
from sqlparser import parse_sql

In [2]:
# 测试函数
sql = "SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department GROUP BY name ORDER BY amount"
print(parse_sql(sql))

{'select_fields': ['max(budget_in_billions)', 'min(budget_in_billions)'], 'tables': ['department'], 'conditions': [], 'group_by': ['name'], 'order_by': ['amount'], 'having': []}


In [3]:
import json

def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def preprocess_data(data):
    inputs = []
    outputs = []
    
    for item in data:
        # 将SQL字典转换为文本
        sql_text = " ".join([" ".join(item['parsed_sql'][key]) for key in item['parsed_sql']])
        inputs.append(sql_text)
        
        # 拼接所有的问题作为输出
        questions = " ".join([step['question'] for step in item['interactive_steps']])
        outputs.append(questions)

    return inputs, outputs

file_path = 'training_set.json'
data = load_dataset(file_path)
inputs, outputs = preprocess_data(data)

In [4]:
from collections import Counter

def build_vocab(texts):
    counter = Counter()
    for text in texts:
        counter.update(text.split())
    return {word: i+1 for i, word in enumerate(counter)}  # 从1开始编号

# 创建词汇表
vocab = build_vocab(inputs + outputs)
vocab_size = len(vocab) + 1  # 加1是因为0通常用于padding

print(f"Vocab Size: {vocab_size}")

Vocab Size: 141


In [5]:
import tensorflow as tf
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)

        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)

        output = tf.reshape(output, (-1, output.shape[2]))

        x = self.fc(output)

        return x, state, attention_weights

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

# def loss_function(real, pred):
#     mask = tf.math.logical_not(tf.math.equal(real, 0))
#     loss_ = loss_object(real, pred)

#     mask = tf.cast(mask, dtype=loss_.dtype)
#     loss_ *= mask

#     return tf.reduce_mean(loss_)

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    print("Mask:", mask.numpy())  # 打印掩码
    print("Unreduced loss:", loss_.numpy())  # 打印未缩减的损失

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    reduced_loss = tf.reduce_mean(loss_)
    print("Reduced loss:", reduced_loss.numpy())  # 打印缩减后的损失
    return reduced_loss


optimizer = tf.keras.optimizers.Adam()

In [7]:
# @tf.function
# def train_step(inp, targ, enc_hidden):
#     loss = 0

#     with tf.GradientTape() as tape:
#         enc_output, enc_hidden = encoder(inp, enc_hidden)

#         dec_hidden = enc_hidden

#         dec_input = tf.expand_dims([vocab['<start>']] * BATCH_SIZE, 1)

#         for t in range(1, targ.shape[1]):
#             predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

#             loss += loss_function(targ[:, t], predictions)

#             dec_input = tf.expand_dims(targ[:, t], 1)

#     batch_loss = (loss / int(targ.shape[1]))

#     variables = encoder.trainable_variables + decoder.trainable_variables

#     gradients = tape.gradient(loss, variables)

#     optimizer.apply_gradients(zip(gradients, variables))

#     return batch_loss
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([tokenizer.word_index['start']] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            # 打印预测和目标值的示例
            if t == 1:
                print("Sample predictions:", predictions[0])
                print("Sample target:", targ[0])

            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    print("Step loss:", batch_loss.numpy())  # 打印每步的损失
    return batch_loss

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
# 清除原有的特殊标记（如果有的话）
inputs = [text.replace('<start> ', '').replace(' <end>', '') for text in inputs]
outputs = [text.replace('<start> ', '').replace(' <end>', '') for text in outputs]

# 重新添加特殊标记
inputs = ['start ' + text + ' end' for text in inputs]
outputs = ['start ' + text + ' end' for text in outputs]

# 创建新的Tokenizer实例
tokenizer = Tokenizer(oov_token='<unk>')
tokenizer.fit_on_texts(inputs + outputs)

# 将文本转换为序列
input_seqs = tokenizer.texts_to_sequences(inputs)
output_seqs = tokenizer.texts_to_sequences(outputs)

# 获取最长序列长度
max_length_input = max(len(seq) for seq in input_seqs)
max_length_output = max(len(seq) for seq in output_seqs)

# 序列填充
input_seqs = pad_sequences(input_seqs, maxlen=max_length_input, padding='post')
output_seqs = pad_sequences(output_seqs, maxlen=max_length_output, padding='post')

# 设置批处理大小
BATCH_SIZE = 64

In [9]:
# 将数据转换为TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((input_seqs, output_seqs))
dataset = dataset.shuffle(len(input_seqs)).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# 设置模型参数
embedding_dim = 128
units = 256
vocab_size = len(tokenizer.word_index) + 1  # 由于使用了tokenizer，词汇表大小根据tokenizer的词汇表确定

# 实例化编码器和解码器
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_size, embedding_dim, units)

EPOCHS = 10
steps_per_epoch = len(input_seqs) // BATCH_SIZE
steps_per_epoch = 2

for epoch in range(EPOCHS):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')

Epoch 1 Loss 0.0000
Epoch 2 Loss 0.0000
Epoch 3 Loss 0.0000
Epoch 4 Loss 0.0000
Epoch 5 Loss 0.0000
Epoch 6 Loss 0.0000
Epoch 7 Loss 0.0000
Epoch 8 Loss 0.0000
Epoch 9 Loss 0.0000
Epoch 10 Loss 0.0000


In [11]:
def evaluate_parsed_sql(parsed_sql):
    # 将parsed_sql转换为文本，并添加特殊标记
    sql_text = "start " + " ".join([" ".join(parsed_sql[key]) for key in parsed_sql]) + " end"

    # Tokenize输入
    inputs = [tokenizer.word_index.get(word, tokenizer.word_index['<unk>']) for word in sql_text.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_input, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    # 模型预测
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tokenizer.word_index['start']], 0)

    for t in range(max_length_output):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += tokenizer.index_word.get(predicted_id, '') + ' '

        if tokenizer.index_word.get(predicted_id) == 'end':
            return result

        dec_input = tf.expand_dims([predicted_id], 0)

    return result


In [12]:
sample_parsed_sql = {
    "parsed_sql": {
        "select_fields": ["name", "email"],
        "tables": ["employees"],
        "conditions": ["department = 'Sales'"]
    }
}

prediction = evaluate_parsed_sql(sample_parsed_sql['parsed_sql'])
print('Predicted translation:', prediction)

Predicted translation: department records' include 'employee groups than and 'customer where department records' include 'employee groups than and 'customer where department records' include 'employee groups than and 'customer where department records' include 'employee groups than and 'customer where department records' include 'employee groups than and 'customer where department records' include 'employee groups than and 'customer 
